In [1]:
%load_ext autoreload
%autoreload 2


import torch

import torch.nn as nn
from torch.distributions import Categorical, Poisson, MixtureSameFamily
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
from torch_perturb.torch_pert_topk import PerturbedTopK

2024-07-05 16:38:27.692629: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-05 16:38:27.745642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 16:38:27.745664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 16:38:27.746764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 16:38:27.754471: I tensorflow/core/platform/cpu_feature_guar

In [2]:
seed=360
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
num_components=4
K=4

In [3]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)
train_X_THS, train_y_TS = to_numpy(train_dataset)

2024-07-05 16:40:44.254002: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-05 16:40:44.254035: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2024-07-05 16:40:44.254042: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: p1cmp078.pax.tufts.edu
2024-07-05 16:40:44.254085: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-07-05 16:40:44.254113: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-07-05 16:40:44.254119: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [4]:

class MixtureOfPoissonsModel(nn.Module):
    def __init__(self, num_components=4, S=12):
        super(MixtureOfPoissonsModel, self).__init__()
        self.num_components = num_components
        self.S = S
        
        # Initialize the log rates and mixture probabilities as learnable parameters
        self.log_poisson_rates = nn.Parameter(torch.rand(num_components))  # Initialize log rates
        self.mixture_probs = nn.Parameter(torch.rand(S, num_components))  # Initialize probabilities

    def params_to_single_tensor(self):
        return torch.cat([self.log_poisson_rates, self.mixture_probs.view(-1)])
    
    def single_tensor_to_params(self, single_tensor):
        log_poisson_rates = single_tensor[:self.num_components]
        mixture_probs = single_tensor[self.num_components:].view(self.S, self.num_components)
        return log_poisson_rates, mixture_probs
    
    def build_from_single_tensor(self, single_tensor):
        log_poisson_rates, mixture_probs = self.single_tensor_to_params(single_tensor)
        poisson_rates = torch.exp(log_poisson_rates)
        mixture_probs_normalized = torch.nn.functional.softmax(mixture_probs, dim=1)
        categorical_dist = Categorical(mixture_probs_normalized)
        expanded_rates = poisson_rates.expand(self.S, self.num_components)
        poisson_dist = Poisson(expanded_rates)
        mixture_dist = MixtureSameFamily(categorical_dist, poisson_dist)
        return mixture_dist
        
    def forward(self):
        # Transform log rates to rates
        poisson_rates = torch.exp(self.log_poisson_rates)
        
        # Normalize mixture_probs to sum to 1 across the components
        mixture_probs_normalized = torch.nn.functional.softmax(self.mixture_probs, dim=1)
        
        # Create the Categorical distribution with the normalized probabilities
        categorical_dist = Categorical(mixture_probs_normalized)
        
        # Expand the Poisson rates to match the number of samples
        expanded_rates = poisson_rates.expand(self.S, self.num_components)
        
        # Create the Poisson distribution with the expanded rates
        poisson_dist = Poisson(expanded_rates)
        
        # Create the MixtureSameFamily distribution
        mixture_dist = MixtureSameFamily(categorical_dist, poisson_dist)
        
        
        
        return mixture_dist

In [5]:
# Instantiate the model
model = MixtureOfPoissonsModel()

# Define an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
mix_model = model()


In [7]:
M =  1000

In [8]:
train_T = train_y_TS.shape[0]
y_sample_TMS = mix_model.sample((train_T, M))

In [9]:
y_sample_TMS.shape

torch.Size([300, 1000, 12])

In [10]:
ratio_rating_TMS = y_sample_TMS/y_sample_TMS.sum(dim=-1, keepdim=True)
ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
ratio_rating_TS.requires_grad_(True)

tensor([[0.0820, 0.0904, 0.0860,  ..., 0.0808, 0.0801, 0.0801],
        [0.0829, 0.0896, 0.0841,  ..., 0.0799, 0.0824, 0.0830],
        [0.0828, 0.0921, 0.0836,  ..., 0.0806, 0.0859, 0.0804],
        ...,
        [0.0812, 0.0911, 0.0838,  ..., 0.0792, 0.0814, 0.0813],
        [0.0799, 0.0913, 0.0830,  ..., 0.0780, 0.0843, 0.0830],
        [0.0798, 0.0894, 0.0857,  ..., 0.0818, 0.0802, 0.0806]],
       requires_grad=True)

In [11]:
def get_log_probs_baked(param):
    distribution = model.build_from_single_tensor(param)
    log_probs_TMS = distribution.log_prob(y_sample_TMS)

    return log_probs_TMS

In [12]:
log_probs_TMS = get_log_probs_baked(model.params_to_single_tensor())

In [13]:
jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, (model.params_to_single_tensor()), strategy='forward-mode', vectorize=True)

In [14]:
score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)

In [15]:
perturbed_top_K_func = PerturbedTopK(k=4)

In [17]:
def get_tensor_loss(rating_TS):
    return -torch_bpr_uncurried(rating_TS, torch.tensor(train_y_TS), K=4, perturbed_top_K_func=perturbed_top_K_func)

In [20]:
# get gradient of negative bpr_t  with respect to ratio rating_TS
positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=4, perturbed_top_K_func=perturbed_top_K_func)
negative_bpr = torch.sum(-positive_bpr_T)
negative_bpr.backward()

In [22]:
loss_grad_TS = ratio_rating_TS.grad

In [24]:
loss_grad_TS

tensor([[  3.8771,  -0.6067,   1.4624,  ...,   4.2596, -15.6125,   1.2657],
        [ -6.0139,  -4.6110,  -4.6700,  ...,   2.1272,   2.9780,   4.4427],
        [ -1.2323,  -0.6200,  -1.5073,  ...,   4.6068,   5.2005,   1.9083],
        ...,
        [ -0.9302,   4.1008,   1.9185,  ...,   1.5544,   2.7539, -20.6214],
        [  1.1973,  -1.3032,  -3.1882,  ...,   4.8600,   4.1975,   7.4375],
        [ -0.3941,  -0.7138,  -3.3775,  ...,   4.3494,   2.1470,   3.7443]])

In [51]:
jac_loss_TS = torch.autograd.functional.jacobian(get_tensor_loss, (ratio_rating_TS), vectorize=True,  strategy='forward-mode')

RuntimeError: vmap: We do not yet support calling random operations inside of vmap. Please perform random operations outside of vmap as a workaround

In [ ]:
from torch.func import vmap, jacrev, functional_call

torch.Size([300, 300, 12])

In [40]:
# get gradient of negative bpr_t  with respect to ratio rating_TS
negative_bpr_T = -positive_bpr_T
#negative_bpr_T.backward()
grad_ratio_rating_TS = torch.autograd.grad(negative_bpr_T, ratio_rating_TS, is_grads_batched=True)

RuntimeError: grad can be implicitly created only for scalar outputs

In [41]:
negative_bpr_T.shape

torch.Size([300])

In [22]:
ratio_rating_TS

tensor([[0.0839, 0.0871, 0.0814,  ..., 0.0818, 0.0816, 0.0832],
        [0.0802, 0.0826, 0.0838,  ..., 0.0824, 0.0844, 0.0802],
        [0.0861, 0.0833, 0.0788,  ..., 0.0863, 0.0869, 0.0837],
        ...,
        [0.0843, 0.0804, 0.0833,  ..., 0.0818, 0.0823, 0.0851],
        [0.0804, 0.0811, 0.0860,  ..., 0.0800, 0.0853, 0.0861],
        [0.0797, 0.0864, 0.0806,  ..., 0.0852, 0.0834, 0.0780]])

In [47]:
from torch.func import vmap, jacrev, functional_call

In [50]:
vmap(jacrev(get_tensor_loss))(ratio_rating_TS)

RuntimeError: In order to use an autograd.Function with functorch transforms (vmap, grad, jvp, jacrev, ...), it must override the setup_context staticmethod. For more details, please see https://pytorch.org/docs/master/notes/extending.func.html

In [16]:
def torch_bpr_uncurried(y_pred, y_true, K=4, perturbed_top_K_func=None):

    top_K_ids = perturbed_top_K_func(y_pred)
    # Sum over k dim
    top_K_ids = top_K_ids.sum(dim=-2)

    true_top_K_val, _  = torch.topk(y_true, K) 
    denominator = torch.sum(true_top_K_val, dim=-1)
    numerator = torch.sum(top_K_ids * y_true, dim=-1)
    bpr = numerator/denominator

    return bpr

In [48]:
model.named_parameters()

<generator object Module.named_parameters at 0x2b8c102fd440>